# Horovod を使った分散学習
Horovod は MPI を使った分散学習のためのフレームワークです。より詳細な情報は [Horovod README](https://github.com/uber/horovod) をご確認下さい。

Amazon SageMaker では Horovod の活用もスクリプトを少し変更するだけで可能です。

## Horovod による分散学習を行う学習スクリプトの作成

#### **オリジナルの学習スクリプトである`training_script/cifar10_keras_sm.py`をコピーした上で、`training_script/cifar10_keras_dist.py.`として保存して下さい。新しいファイルを書き換え用のファイルとして使います。**


#### ① Horovod の初期設定

Horovod を使用するように `main()` 関数の中に 下記を追加します。



```python
    import horovod.keras as hvd
    hvd.init()
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    config.gpu_options.visible_device_list = str(hvd.local_rank())
    K.set_session(tf.Session(config=config))
```

#### ② Callbacks の設定

Horovod へ対応するために `main()` 関数の中に callbacks を追加します。

```python
    callbacks.append(hvd.callbacks.BroadcastGlobalVariablesCallback(0))
    callbacks.append(hvd.callbacks.MetricAverageCallback())
    callbacks.append(hvd.callbacks.LearningRateWarmupCallback(warmup_epochs=5, verbose=1))
```

また、checkpoint と tensorboard の callback がシングルプロセスのログだけ送信するように変更します。 

```python
    if hvd.rank() == 0:
        callbacks.append(ModelCheckpoint(args.model_output_dir + '/checkpoint-{epoch}.h5'))
        callbacks.append(TensorBoard(log_dir=args.model_output_dir,update_freq='epoch'))
```

#### ③ Horovod に対応した Optimizer の設定変更
Horovod へ対応するために keras_model_fn へ `hvd` 引数を追加します。

```python
#  hvd を追加します。
def keras_model_fn(learning_rate, weight_decay, optimizer, momentum, hvd): 
```

その、`keras_model_fn()` 関数の中において`size=1` を `size=hvd.size()`　へ変更。
さらに、
```python
 model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
```
の直前に

```python
opt = hvd.DistributedOptimizer(opt)
```
を追加します。


最後に、`main()` 関数の中で、model インスタンスを作成する際に、`hvd` を引数に渡すよう書き換えます。
```python
model = keras_model_fn(args.learning_rate, args.weight_decay, args.optimizer, args.momentum, hvd)
```

## 分散学習の実行
下記の設定を Estimator オブジェクトに渡すことで Horovod で分散学習についての設定をすることができます。

```python
distributions = {'mpi': {
                    'enabled': True,
                    'processes_per_host': # それぞれのホストでのインスタンスの数
                        }
                }
```

In [ ]:
import os
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()
role = get_execution_role()

# dataset_location = sagemaker_session.upload_data(path='data', key_prefix='data/DEMO-cifar10')
# display(dataset_location)

今回は各ホストで1プロセスが実行されるような設定としています。

In [ ]:
from sagemaker.tensorflow import TensorFlow

distributions = {'mpi': {
                    'enabled': True,
                    'processes_per_host': 1}
                }

# Change base_job_name to 'cifar10-dist' for console visibility
estimator = TensorFlow(base_job_name='cifar10-dist',
                       entry_point='cifar10_keras_dist.py',
                       source_dir='training_script',
                       role=role,
                       framework_version='1.12.0',
                       py_version='py3',
                       hyperparameters={'epochs' : 10},
                       train_instance_count=2,
                       train_instance_type='ml.p2.xlarge',
                       distributions=distributions
                      )

In [ ]:
estimator.fit({'train':'{}/train'.format(dataset_location),
              'validation':'{}/validation'.format(dataset_location),
              'eval':'{}/eval'.format(dataset_location)})